In [1]:
import os
import cv2
import json
import random
import colorsys

import numpy as np
import tensorflow as tf

from keras.models import Sequential, Model ,model_from_yaml
from keras.activations import linear
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adagrad
from keras.utils import plot_model
from keras.engine.topology import Layer
from yolo3.model import yolo_eval
from yolo3.utils import letterbox_image
from keras import backend as K

Using TensorFlow backend.


In [2]:
def _get_class():
    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names

In [3]:
def _get_anchors():
    with open(anchors_path) as f:
        anchors = f.readline()
        anchors = [float(x) for x in anchors.split(',')]
        anchors = np.array(anchors).reshape(-1, 2)
    return anchors

In [4]:
def generate():
    assert model_path.endswith('.h5'), 'Keras model must be a .h5 file.'
    yaml_file = open('infer_model.yaml', 'r')
    loaded_model_yaml = yaml_file.read()
    yaml_file.close()
    Infer_model = model_from_yaml(loaded_model_yaml)
    print('{} model, anchors, and classes loaded.'.format(model_path))

    # Generate colors for drawing bounding boxes.
    hsv_tuples = [(x / len(class_names), 1., 1.)
                  for x in range(len(class_names))]
    colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
    colors = list(
        map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)),
            colors))
    random.seed(10101)  # Fixed seed for consistent colors across runs.
    # Shuffle colors to decorrelate adjacent classes.
    random.shuffle(colors)
    random.seed(None)  # Reset seed to default.

    # Generate output tensor targets for filtered bounding boxes.
    input_image_shape = K.placeholder(shape=(2, ))
    boxes, scores, classes = yolo_eval(Infer_model.output, anchors,
                                       len(class_names), input_image_shape,
                                       score_threshold=score, iou_threshold=iou)
    return boxes, scores, classes

In [5]:
def close_session(self):
    sess.close()

In [6]:
def detect_image(image, animals=1, predictEdge=0, pos=-1, database=None):
    predictEdge /= 100
    if is_fixed_size:
        assert model_image_size[0] % 32 == 0, 'Multiples of 32 required'
        assert model_image_size[1] % 32 == 0, 'Multiples of 32 required'
        boxed_image = letterbox_image(
            image, tuple(reversed(model_image_size)))
    else:
        new_image_size = (image.width - (image.width % 32),
                          image.height - (image.height % 32))
        boxed_image = letterbox_image(image, new_image_size)
    image_data = np.array(boxed_image, dtype='float32')

    # print(image_data.shape)
    image_data /= 255.
    image_data = np.expand_dims(image_data, 0)  # Add batch dimension.

    out_boxes, out_scores, out_classes = sess.run(
        [boxes, scores, classes],
        feed_dict={
            Infer_model.input: image_data,
            input_image_shape: [image.size[1], image.size[0]],
            K.learning_phase(): 0
        })

    # print('Found {} boxes for {}'.format(len(out_boxes), 'img'))

    font = ImageFont.truetype(font='font/FiraMono-Medium.otf',
                              size=np.floor(3e-2 * image.size[1] + 0.5).astype('int32'))
    thickness = (image.size[0] + image.size[1]) // 300

    for i, c in reversed(list(enumerate(out_classes))):
        predicted_class = class_names[c]
        box = out_boxes[i]
        score = out_scores[i]
        score = float(score)
        score = round(score, 2)
        # print(score, predicted_class, c)
        if((score < predictEdge) or ((animals >> c) & 1 == 0)):
            continue
        label = '{} {:.2f}'.format(predicted_class, score)
        draw = ImageDraw.Draw(image)
        label_size = draw.textsize(label, font)

        top, left, bottom, right = box
        top = max(0, np.floor(top + 0.5).astype('int32'))
        left = max(0, np.floor(left + 0.5).astype('int32'))
        bottom = min(image.size[1], np.floor(bottom + 0.5).astype('int32'))
        right = min(image.size[0], np.floor(right + 0.5).astype('int32'))
        # print(label, (left, top), (right, bottom), pos, c, type(top))
        if(pos != -1):
            database.insert({"pos": pos, "left": int(left), "right": int(right),
                             "top": int(top), "bottom": int(bottom), "predict": int(c), "score": score})
        if top - label_size[1] >= 0:
            text_origin = np.array([left, top - label_size[1]])
        else:
            text_origin = np.array([left, top + 1])

        # My kingdom for a good redistributable image drawing library.
        for i in range(thickness):
            draw.rectangle(
                [left + i, top + i, right - i, bottom - i],
                outline=colors[c])
        draw.rectangle(
            [tuple(text_origin), tuple(text_origin + label_size)],
            fill=colors[c])
        draw.text(text_origin, label, fill=(0, 0, 0), font=font)
        del draw

    # end = time.time()
    # print(end - start)
    return image

In [7]:
model_path   = 'model_data/yolo.h5'
anchors_path = 'model_data/yolo_anchors.txt'
classes_path = 'model_data/coco_classes.txt'
score = 0.3
iou = 0.5
class_names = _get_class()
anchors = _get_anchors()
sess = K.get_session()
model_image_size = (416, 416)  # fixed size or (None, None)
is_fixed_size = model_image_size != (None, None)
boxes, scores, classes = generate()

model_data/yolo.h5 model, anchors, and classes loaded.
